In [1]:
import sys, os
sys.path.append('/cr/users/filip/bin/')

from utils.Auger.SD import Monitoring

from utils.binaries import *
from utils.plotting import *

12:36:20 [INFO] utils          -- ( +3141ms) -- import logging
12:36:20 [INFO] utils.binaries -- (  +183ms) -- import numpy as np
12:36:21 [INFO] utils.binaries -- (  +393ms) -- import pandas as pd
12:36:21 [INFO] utils.binaries -- (    +4ms) -- import binaries.tools as binaries
12:36:22 [INFO] utils.plotting -- (  +887ms) -- import matplotlib.pyplot as plt
12:36:22 [INFO] utils.plotting -- (  +496ms) -- import seaborn as sns
12:36:22 [DBUG] utils.plotting -- (   +15ms) -- font size set to 4
12:36:22 [DBUG] utils.plotting -- (    +1ms) -- figure size set to [2.4, 1.2]
12:36:22 [DBUG] utils.plotting -- (    +1ms) -- plotting style set to `notebook`.
12:36:22 [INFO] utils.plotting -- (    +3ms) -- import plotting.tools as plot


# Where calibration histogram?

In [6]:
MonitTest = Monitoring.Monit(2023, 12, 11)

print(MonitTest)

File: /cr/auger02/Prod/monit/Sd//2023/12/mc_2023_12_11_00h00.root
on branch: SDMonCal/SDMonCalBranch
  -: TObject
    -: fUniqueID
    -: fBits
  -: fLsId
  -: fTime
  -: fCDASTime
  -: fMonitoring
    -: fIsMonitoring
    -: fIsUUB
    -: fPMV[3]
    -: fPMI[3]
    -: fPMT[3]
    -: fElectT
    -: fBatteryT[2]
    -: fBatteryV[2]
    -: fCurrentLoad
    -: fSolarPanelV
    -: fSolarPanelI
    -: fWaterLevel
    -: fWaterT
    -: f12VRadio
    -: f3V
    -: f_3V
    -: f5V
    -: f12V
    -: f24V
    -: fADCBaseline
    -: fDAC4Voltage
    -: f3VAnalogPower
    -: f12VMultiplexer
    -: fUPMV[3]
    -: fUPMI[3]
    -: fUPMT[3]
    -: fExtT
    -: fElectP
    -: fElectH
    -: fADC[9]
    -: f12V_WT
    -: f12VPMT[3]
    -: f12VPM
    -: f24VExt[2]
    -: f3V3
    -: f5VGPS
    -: f3V3Analog
    -: f_3V3Analog
    -: f5VUSB
    -: f1V0
    -: f1V2
    -: f1V8
    -: f10V
    -: f12VRadioI
    -: f12VPMI
    -: f3V3AnalogI
    -: f_3V3AnalogI
    -: f5VGPSI
    -: f1V0I
    -: f1V2I
    

In [7]:
import uproot

file = uproot.open('/cr/auger02/Prod/SDdata/2023/11/sd_2023_11_01_10h58.root')

In [9]:
print(file[file.keys()[0]])

NotImplementedError: memberwise serialization of AsVector(IoSdStation)
    in file /cr/auger02/Prod/SDdata/2023/11/sd_2023_11_01_10h58.root

In [2]:
import ROOT

Welcome to JupyROOT 6.24/00


In [5]:
myFile = ROOT.TFile.Open("/cr/auger02/Prod/SDdata/2023/11/sd_2023_11_01_10h58.root")
myFile.Get("74122430#0#2C1&3C2&4C4#1382913684#5")